<a href="https://colab.research.google.com/github/MarcoBegnozzi/Project0/blob/master/Agent3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

V1 gnerate a course

In [1]:
# Course Generator AI Agent for Google Colab
# This agent generates university course titles and descriptions

import requests
from typing import Dict, List, Optional
import json
import re
from datetime import datetime

class CourseGeneratorAgent:
    def __init__(self):
        self.name = "Course Generator"
        self.description = "Generates university course titles and descriptions"
        self.capabilities = ["topic_research", "course_generation", "web_search"]

    def search_topic_info(self, topic: str) -> str:
        """
        Retrieve information about a topic from the internet
        Using DuckDuckGo Instant Answer API (free, no API key needed)
        """
        try:
            # DuckDuckGo Instant Answer API
            url = f"https://api.duckduckgo.com/?q={topic}&format=json&no_html=1&skip_disambig=1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()

                # Extract relevant information
                abstract = data.get('Abstract', '')
                definition = data.get('Definition', '')

                # Get related topics
                related_topics = []
                if 'RelatedTopics' in data:
                    for item in data['RelatedTopics'][:3]:  # Limit to 3
                        if isinstance(item, dict) and 'Text' in item:
                            related_topics.append(item['Text'])

                # Combine information
                info = []
                if abstract:
                    info.append(f"Overview: {abstract}")
                if definition:
                    info.append(f"Definition: {definition}")
                if related_topics:
                    info.append(f"Related areas: {'; '.join(related_topics)}")

                return " | ".join(info) if info else f"Basic information about {topic}"

        except Exception as e:
            print(f"Web search failed: {e}")

        return f"General topic: {topic}"

    def generate_course_content(self, topic: str, topic_info: str = "") -> Dict[str, str]:
        """
        Generate course title and description using rule-based approach
        This simulates what an LLM would do but works without external models
        """

        # Course title templates
        title_templates = [
            f"Introduction to {topic}",
            f"Fundamentals of {topic}",
            f"Advanced {topic}",
            f"{topic}: Theory and Practice",
            f"Modern {topic}",
            f"{topic} in Contemporary Context",
            f"Principles of {topic}",
            f"{topic} and Its Applications"
        ]

        # Select title based on topic characteristics
        topic_lower = topic.lower()
        if any(word in topic_lower for word in ['advanced', 'complex', 'deep']):
            title = f"Advanced {topic}"
        elif any(word in topic_lower for word in ['basic', 'intro', 'fundamental']):
            title = f"Introduction to {topic}"
        else:
            title = f"Fundamentals of {topic}"

        # Generate description components
        description_parts = []

        # Opening statement
        description_parts.append(f"This course provides a comprehensive examination of {topic.lower()}")

        # Add context from web search if available
        if topic_info and "Overview:" in topic_info:
            overview = topic_info.split("Overview:")[1].split("|")[0].strip()
            if overview:
                description_parts.append(f"exploring {overview.lower()}")

        # Learning objectives (generic but adaptable)
        objectives = [
            f"understand the fundamental principles of {topic.lower()}",
            f"analyze key concepts and methodologies in {topic.lower()}",
            f"evaluate contemporary applications and implications",
            "develop critical thinking skills in the field",
            "engage with current research and developments"
        ]

        description_parts.append("Students will " + ", ".join(objectives[:3]))

        # Course structure
        description_parts.append(
            "The course combines theoretical foundations with practical applications, "
            "incorporating case studies, interactive discussions, and hands-on exercises."
        )

        # Prerequisites and target audience
        description_parts.append(
            "Suitable for undergraduate students with basic academic preparation. "
            "No prior specialized knowledge required."
        )

        # Join all parts
        description = ". ".join(description_parts) + "."

        return {
            "title": title,
            "description": description,
            "topic": topic,
            "generated_at": datetime.now().isoformat(),
            "source_info": topic_info[:100] + "..." if len(topic_info) > 100 else topic_info
        }

    def process_request(self, topic: str, use_web_search: bool = True) -> Dict[str, str]:
        """
        Main method to process a topic and generate course content
        """
        print(f"🤖 Agent '{self.name}' processing topic: {topic}")

        # Step 1: Research topic if requested
        topic_info = ""
        if use_web_search:
            print("🔍 Searching for topic information...")
            topic_info = self.search_topic_info(topic)
            print(f"✅ Found: {topic_info[:100]}...")

        # Step 2: Generate course content
        print("🎓 Generating course content...")
        course_data = self.generate_course_content(topic, topic_info)

        print("✅ Course generation completed!")
        return course_data

    def display_result(self, course_data: Dict[str, str]):
        """
        Display the generated course in a formatted way
        """
        print("\n" + "="*60)
        print("📚 GENERATED UNIVERSITY COURSE")
        print("="*60)
        print(f"📖 TITLE: {course_data['title']}")
        print(f"🎯 TOPIC: {course_data['topic']}")
        print(f"🕒 GENERATED: {course_data['generated_at']}")
        print("\n📝 DESCRIPTION:")
        print("-" * 40)
        print(course_data['description'])
        print("\n🔍 SOURCE INFO:")
        print("-" * 40)
        print(course_data['source_info'])
        print("="*60)

# Example usage and testing
def main():
    """
    Main function to demonstrate the agent
    """
    # Initialize the agent
    agent = CourseGeneratorAgent()

    # Test topics
    test_topics = [
        "Machine Learning",
        "Sustainable Energy",
        "Digital Marketing",
        "Cognitive Psychology",
        "Blockchain Technology"
    ]

    print("🚀 Starting Course Generator Agent Demo")
    print("This agent can work with any topic!\n")

    # Interactive mode
    while True:
        print("\n" + "="*50)
        print("COURSE GENERATOR AGENT")
        print("="*50)
        print("1. Enter a custom topic")
        print("2. Choose from example topics")
        print("3. Generate without web search")
        print("4. Exit")

        choice = input("\nSelect option (1-4): ").strip()

        if choice == "1":
            topic = input("Enter your topic: ").strip()
            if topic:
                course_data = agent.process_request(topic, use_web_search=True)
                agent.display_result(course_data)

        elif choice == "2":
            print("\nExample topics:")
            for i, topic in enumerate(test_topics, 1):
                print(f"{i}. {topic}")

            try:
                topic_choice = int(input("Select topic (1-5): ")) - 1
                if 0 <= topic_choice < len(test_topics):
                    topic = test_topics[topic_choice]
                    course_data = agent.process_request(topic, use_web_search=True)
                    agent.display_result(course_data)
                else:
                    print("Invalid selection!")
            except ValueError:
                print("Please enter a valid number!")

        elif choice == "3":
            topic = input("Enter your topic (no web search): ").strip()
            if topic:
                course_data = agent.process_request(topic, use_web_search=False)
                agent.display_result(course_data)

        elif choice == "4":
            print("👋 Goodbye! Agent shutting down.")
            break

        else:
            print("Invalid option! Please try again.")

# For Colab: Run this cell to start the agent
if __name__ == "__main__":
    main()

# Quick test function for Colab
def quick_test():
    """Quick test without interactive mode - good for Colab"""
    agent = CourseGeneratorAgent()

    # Test with a sample topic
    topic = "Artificial Intelligence Ethics"
    print(f"🧪 Quick test with topic: {topic}")

    course_data = agent.process_request(topic)
    agent.display_result(course_data)

    return course_data

# Uncomment the line below to run a quick test
quick_test()

🚀 Starting Course Generator Agent Demo
This agent can work with any topic!


COURSE GENERATOR AGENT
1. Enter a custom topic
2. Choose from example topics
3. Generate without web search
4. Exit

Select option (1-4): 2

Example topics:
1. Machine Learning
2. Sustainable Energy
3. Digital Marketing
4. Cognitive Psychology
5. Blockchain Technology
Select topic (1-5): 2
🤖 Agent 'Course Generator' processing topic: Sustainable Energy
🔍 Searching for topic information...
✅ Found: Overview: Energy is sustainable if it "meets the needs of the present without compromising the abili...
🎓 Generating course content...
✅ Course generation completed!

📚 GENERATED UNIVERSITY COURSE
📖 TITLE: Fundamentals of Sustainable Energy
🎯 TOPIC: Sustainable Energy
🕒 GENERATED: 2025-06-21T08:02:48.635756

📝 DESCRIPTION:
----------------------------------------
This course provides a comprehensive examination of sustainable energy. exploring energy is sustainable if it "meets the needs of the present without compro

{'title': 'Fundamentals of Artificial Intelligence Ethics',
 'description': 'This course provides a comprehensive examination of artificial intelligence ethics. exploring the ethics of artificial intelligence covers a broad range of topics within ai that are considered to have particular ethical stakes. this includes algorithmic biases, fairness, automated decision-making, accountability, privacy, and regulation. it also covers various emerging or potential future challenges such as machine ethics, lethal autonomous weapon systems, arms race dynamics, ai safety and alignment, technological unemployment, ai-enabled misinformation, how to treat certain ai systems if they have a moral status, artificial superintelligence and existential risks. some application areas may also have particularly important ethical implications, like healthcare, education, criminal justice, or the military.. Students will understand the fundamental principles of artificial intelligence ethics, analyze key conc

v2 generate a course, save and check before generating

In [3]:
# Enhanced Course Generator AI Agent with Google Drive Integration
# This agent generates university courses and manages them in Google Drive

import requests
from typing import Dict, List, Optional, Tuple
import json
import re
from datetime import datetime
import os
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
from difflib import SequenceMatcher

class EnhancedCourseGeneratorAgent:
    def __init__(self, drive_folder_name: str = "AI_Generated_Courses"):
        self.name = "Enhanced Course Generator"
        self.description = "Generates university courses with Google Drive integration"
        self.capabilities = ["topic_research", "course_generation", "web_search", "drive_integration", "duplicate_detection"]
        self.drive_folder_name = drive_folder_name
        self.drive_service = None
        self.folder_id = None
        self.existing_courses = []

        # Initialize Google Drive
        self._setup_drive()

    def _setup_drive(self):
        """Setup Google Drive API connection"""
        try:
            print("🔧 Setting up Google Drive connection...")

            # Mount Google Drive
            drive.mount('/content/drive')

            # Authenticate and build service
            auth.authenticate_user()
            self.drive_service = build('drive', 'v3')

            # Create or find the courses folder
            self._setup_courses_folder()

            # Load existing courses
            self._load_existing_courses()

            print("✅ Google Drive setup completed!")

        except Exception as e:
            print(f"❌ Drive setup failed: {e}")
            print("Note: Make sure to run this in Google Colab with Drive access")

    def _setup_courses_folder(self):
        """Create or find the courses folder in Google Drive"""
        try:
            # Search for existing folder
            query = f"name='{self.drive_folder_name}' and mimeType='application/vnd.google-apps.folder'"
            results = self.drive_service.files().list(q=query).execute()
            items = results.get('files', [])

            if items:
                self.folder_id = items[0]['id']
                print(f"📁 Found existing folder: {self.drive_folder_name}")
            else:
                # Create new folder
                folder_metadata = {
                    'name': self.drive_folder_name,
                    'mimeType': 'application/vnd.google-apps.folder'
                }
                folder = self.drive_service.files().create(body=folder_metadata).execute()
                self.folder_id = folder.get('id')
                print(f"📁 Created new folder: {self.drive_folder_name}")

        except Exception as e:
            print(f"❌ Folder setup failed: {e}")

    def _load_existing_courses(self):
        """Load all existing courses from the Drive folder"""
        try:
            if not self.folder_id:
                return

            print("📚 Loading existing courses...")

            # Search for JSON files in the folder
            query = f"'{self.folder_id}' in parents and name contains '.json'"
            results = self.drive_service.files().list(q=query).execute()
            files = results.get('files', [])

            self.existing_courses = []

            for file in files:
                try:
                    # Download and parse each course file
                    request = self.drive_service.files().get_media(fileId=file['id'])
                    file_content = io.BytesIO()
                    downloader = MediaIoBaseDownload(file_content, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()

                    # Parse JSON content
                    file_content.seek(0)
                    course_data = json.loads(file_content.read().decode('utf-8'))
                    self.existing_courses.append(course_data)

                except Exception as e:
                    print(f"⚠️ Could not load course file {file['name']}: {e}")

            print(f"✅ Loaded {len(self.existing_courses)} existing courses")

        except Exception as e:
            print(f"❌ Failed to load existing courses: {e}")

    def _similarity_score(self, text1: str, text2: str) -> float:
        """Calculate similarity between two texts"""
        return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

    def _find_similar_courses(self, topic: str, threshold: float = 0.3) -> List[Dict]:
        """Find courses similar to the given topic"""
        similar_courses = []

        for course in self.existing_courses:
            # Check similarity with topic, title, and description
            topic_sim = self._similarity_score(topic, course.get('topic', ''))
            title_sim = self._similarity_score(topic, course.get('title', ''))

            max_similarity = max(topic_sim, title_sim)

            if max_similarity > threshold:
                course_copy = course.copy()
                course_copy['similarity_score'] = max_similarity
                similar_courses.append(course_copy)

        # Sort by similarity (highest first)
        similar_courses.sort(key=lambda x: x['similarity_score'], reverse=True)
        return similar_courses

    def _generate_alternative_course(self, original_topic: str, similar_courses: List[Dict]) -> Dict[str, str]:
        """Generate an alternative course when similar ones exist"""

        # Analyze existing courses to determine what type of alternative to create
        existing_levels = []
        existing_topics = []

        for course in similar_courses:
            title = course.get('title', '').lower()
            topic = course.get('topic', '').lower()

            # Detect course levels
            if any(word in title for word in ['introduction', 'intro', 'basic', 'fundamentals']):
                existing_levels.append('beginner')
            elif any(word in title for word in ['advanced', 'expert', 'master']):
                existing_levels.append('advanced')
            else:
                existing_levels.append('intermediate')

            existing_topics.append(topic)

        # Decision logic for alternative course
        alternative_type = "advanced"  # default

        if 'beginner' in existing_levels and 'advanced' not in existing_levels:
            alternative_type = "advanced"
        elif 'advanced' in existing_levels and 'beginner' not in existing_levels:
            alternative_type = "beginner"
        elif len(similar_courses) >= 2:
            alternative_type = "specialized"

        return self._create_alternative_course(original_topic, alternative_type, similar_courses)

    def _create_alternative_course(self, topic: str, alt_type: str, similar_courses: List[Dict]) -> Dict[str, str]:
        """Create alternative course based on type"""

        if alt_type == "advanced":
            title = f"Advanced {topic}"
            description_intro = f"This advanced course builds upon foundational knowledge of {topic.lower()}"
            focus_areas = [
                "cutting-edge research and developments",
                "complex theoretical frameworks",
                "advanced methodologies and techniques",
                "critical analysis of contemporary issues",
                "independent research projects"
            ]
            prerequisites = f"Completion of introductory {topic.lower()} course or equivalent experience required."

        elif alt_type == "beginner":
            title = f"Introduction to {topic}"
            description_intro = f"This introductory course provides a gentle entry point into {topic.lower()}"
            focus_areas = [
                "fundamental concepts and terminology",
                "basic principles and applications",
                "historical context and development",
                "practical exercises and examples",
                "foundational skills development"
            ]
            prerequisites = "No prior knowledge required. Suitable for all students."

        else:  # specialized
            specializations = [
                f"{topic} in Healthcare Applications",
                f"{topic} for Business Innovation",
                f"{topic} and Society",
                f"Ethical Considerations in {topic}",
                f"{topic} Research Methods"
            ]
            title = specializations[len(similar_courses) % len(specializations)]
            description_intro = f"This specialized course explores the intersection of {topic.lower()} with specific domain applications"
            focus_areas = [
                "domain-specific applications",
                "interdisciplinary approaches",
                "case studies and real-world examples",
                "professional skills development",
                "industry collaboration projects"
            ]
            prerequisites = f"Basic understanding of {topic.lower()} recommended."

        # Build full description
        description_parts = [
            description_intro,
            f"Students will explore {', '.join(focus_areas[:3])}",
            "The course emphasizes both theoretical understanding and practical application through interactive learning experiences.",
            prerequisites
        ]

        description = ". ".join(description_parts) + "."

        return {
            "title": title,
            "description": description,
            "topic": topic,
            "course_type": alt_type,
            "related_courses": len(similar_courses),
            "generated_at": datetime.now().isoformat(),
            "alternative_reason": f"Similar courses detected - created {alt_type} alternative"
        }

    def search_topic_info(self, topic: str) -> str:
        """Retrieve information about a topic from the internet"""
        try:
            url = f"https://api.duckduckgo.com/?q={topic}&format=json&no_html=1&skip_disambig=1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()

                abstract = data.get('Abstract', '')
                definition = data.get('Definition', '')

                related_topics = []
                if 'RelatedTopics' in data:
                    for item in data['RelatedTopics'][:3]:
                        if isinstance(item, dict) and 'Text' in item:
                            related_topics.append(item['Text'])

                info = []
                if abstract:
                    info.append(f"Overview: {abstract}")
                if definition:
                    info.append(f"Definition: {definition}")
                if related_topics:
                    info.append(f"Related areas: {'; '.join(related_topics)}")

                return " | ".join(info) if info else f"Basic information about {topic}"

        except Exception as e:
            print(f"Web search failed: {e}")

        return f"General topic: {topic}"

    def generate_course_content(self, topic: str, topic_info: str = "") -> Dict[str, str]:
        """Generate standard course content"""

        title = f"Fundamentals of {topic}"

        description_parts = [
            f"This course provides a comprehensive examination of {topic.lower()}",
            f"Students will understand the fundamental principles of {topic.lower()}, analyze key concepts and methodologies, and evaluate contemporary applications",
            "The course combines theoretical foundations with practical applications, incorporating case studies and interactive discussions.",
            "Suitable for undergraduate students with basic academic preparation."
        ]

        description = ". ".join(description_parts) + "."

        return {
            "title": title,
            "description": description,
            "topic": topic,
            "course_type": "standard",
            "generated_at": datetime.now().isoformat(),
            "source_info": topic_info[:100] + "..." if len(topic_info) > 100 else topic_info
        }

    def _save_course_to_drive(self, course_data: Dict[str, str]) -> str:
        """Save course data to Google Drive"""
        try:
            if not self.drive_service or not self.folder_id:
                print("❌ Google Drive not properly configured")
                return None

            # Create filename
            safe_title = re.sub(r'[^\w\s-]', '', course_data['title'])
            safe_title = re.sub(r'[-\s]+', '_', safe_title)
            filename = f"{safe_title}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

            # Create file content
            file_content = json.dumps(course_data, indent=2)

            # Create temporary file path
            temp_file = f"/tmp/{filename}"

            # Write content to temporary file
            with open(temp_file, 'w', encoding='utf-8') as f:
                f.write(file_content)

            # Upload to Drive using the temporary file
            file_metadata = {
                'name': filename,
                'parents': [self.folder_id]
            }

            media = MediaFileUpload(
                temp_file,
                mimetype='application/json',
                resumable=True
            )

            file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media
            ).execute()

            # Clean up temporary file
            if os.path.exists(temp_file):
                os.remove(temp_file)

            print(f"💾 Course saved to Drive: {filename}")
            return file.get('id')

        except Exception as e:
            print(f"❌ Failed to save to Drive: {e}")
            # Clean up temp file in case of error
            temp_file = f"/tmp/{filename if 'filename' in locals() else 'temp_course'}.json"
            if os.path.exists(temp_file):
                os.remove(temp_file)
            return None

    def process_request(self, topic: str, use_web_search: bool = True) -> Dict[str, str]:
        """Main method to process a topic and generate course content"""
        print(f"🤖 Agent '{self.name}' processing topic: {topic}")

        # Step 1: Check for similar existing courses
        print("🔍 Checking for similar existing courses...")
        similar_courses = self._find_similar_courses(topic)

        if similar_courses:
            print(f"📚 Found {len(similar_courses)} similar courses:")
            for course in similar_courses[:3]:  # Show top 3
                print(f"  - {course['title']} (similarity: {course['similarity_score']:.2f})")

            print("🎯 Generating alternative course...")
            course_data = self._generate_alternative_course(topic, similar_courses)
        else:
            print("✨ No similar courses found, generating standard course...")

            # Step 2: Research topic if requested
            topic_info = ""
            if use_web_search:
                print("🔍 Searching for topic information...")
                topic_info = self.search_topic_info(topic)
                print(f"✅ Found: {topic_info[:100]}...")

            # Step 3: Generate course content
            course_data = self.generate_course_content(topic, topic_info)

        # Step 4: Save to Google Drive
        print("💾 Saving course to Google Drive...")
        file_id = self._save_course_to_drive(course_data)
        if file_id:
            course_data['drive_file_id'] = file_id
            # Add to existing courses list
            self.existing_courses.append(course_data)

        print("✅ Course generation completed!")
        return course_data

    def display_result(self, course_data: Dict[str, str]):
        """Display the generated course in a formatted way"""
        print("\n" + "="*60)
        print("📚 GENERATED UNIVERSITY COURSE")
        print("="*60)
        print(f"📖 TITLE: {course_data['title']}")
        print(f"🎯 TOPIC: {course_data['topic']}")
        print(f"🏷️ TYPE: {course_data.get('course_type', 'standard').upper()}")

        if 'alternative_reason' in course_data:
            print(f"💡 REASON: {course_data['alternative_reason']}")

        print(f"🕒 GENERATED: {course_data['generated_at']}")

        if 'drive_file_id' in course_data:
            print(f"💾 SAVED TO DRIVE: ✅")

        print("\n📝 DESCRIPTION:")
        print("-" * 40)
        print(course_data['description'])

        if 'source_info' in course_data:
            print("\n🔍 SOURCE INFO:")
            print("-" * 40)
            print(course_data['source_info'])

        print("="*60)

    def list_existing_courses(self):
        """Display all existing courses"""
        if not self.existing_courses:
            print("📚 No courses found in the database.")
            return

        print(f"\n📚 EXISTING COURSES ({len(self.existing_courses)} total)")
        print("="*60)

        for i, course in enumerate(self.existing_courses, 1):
            print(f"{i}. {course['title']}")
            print(f"   Topic: {course['topic']} | Type: {course.get('course_type', 'standard')}")
            print(f"   Generated: {course['generated_at'][:10]}")
            print()

# Main function for demonstration
def main():
    """Main function to demonstrate the enhanced agent"""
    print("🚀 Starting Enhanced Course Generator Agent")
    print("This agent now integrates with Google Drive!")

    # Initialize the agent (will setup Drive connection)
    agent = EnhancedCourseGeneratorAgent()

    if not agent.drive_service:
        print("❌ Could not connect to Google Drive. Please check your setup.")
        return

    # Interactive mode
    while True:
        print("\n" + "="*50)
        print("ENHANCED COURSE GENERATOR AGENT")
        print("="*50)
        print("1. Generate course for new topic")
        print("2. View existing courses")
        print("3. Test with sample topics")
        print("4. Generate without web search")
        print("5. Exit")

        choice = input("\nSelect option (1-5): ").strip()

        if choice == "1":
            topic = input("Enter your topic: ").strip()
            if topic:
                course_data = agent.process_request(topic, use_web_search=True)
                agent.display_result(course_data)

        elif choice == "2":
            agent.list_existing_courses()

        elif choice == "3":
            test_topics = [
                "Machine Learning",
                "Machine Learning",  # Duplicate to test alternative generation
                "Deep Learning",     # Similar to test alternative generation
                "Sustainable Energy",
                "Digital Marketing"
            ]

            print("🧪 Running test with sample topics...")
            for topic in test_topics:
                print(f"\n--- Processing: {topic} ---")
                course_data = agent.process_request(topic)
                agent.display_result(course_data)
                input("\nPress Enter to continue to next topic...")

        elif choice == "4":
            topic = input("Enter your topic (no web search): ").strip()
            if topic:
                course_data = agent.process_request(topic, use_web_search=False)
                agent.display_result(course_data)

        elif choice == "5":
            print("👋 Goodbye! Agent shutting down.")
            break

        else:
            print("Invalid option! Please try again.")

# Quick test function for Colab
def quick_test():
    """Quick test without interactive mode - good for Colab"""
    print("🧪 Quick Test Mode")

    agent = EnhancedCourseGeneratorAgent()

    if not agent.drive_service:
        print("❌ Could not connect to Google Drive")
        return

    # Test with sample topics
    test_topics = ["Artificial Intelligence", "AI Ethics", "Machine Learning"]

    for topic in test_topics:
        print(f"\n--- Testing with: {topic} ---")
        course_data = agent.process_request(topic)
        agent.display_result(course_data)

    print(f"\n📊 Total courses in database: {len(agent.existing_courses)}")

# For immediate testing
if __name__ == "__main__":
    main()

# Uncomment to run quick test
# quick_test()

🚀 Starting Enhanced Course Generator Agent
This agent now integrates with Google Drive!
🔧 Setting up Google Drive connection...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Found existing folder: AI_Generated_Courses
📚 Loading existing courses...
✅ Loaded 0 existing courses
✅ Google Drive setup completed!

ENHANCED COURSE GENERATOR AGENT
1. Generate course for new topic
2. View existing courses
3. Test with sample topics
4. Generate without web search
5. Exit

Select option (1-5): 3
🧪 Running test with sample topics...

--- Processing: Machine Learning ---
🤖 Agent 'Enhanced Course Generator' processing topic: Machine Learning
🔍 Checking for similar existing courses...
✨ No similar courses found, generating standard course...
🔍 Searching for topic information...
✅ Found: Overview: Machine learning is a field of study in artificial intelligence concerned with the develop...
💾 Saving course to Google Dri

v3 generate a course, save and check before generating, gives trending topics

In [4]:
# Enhanced Course Generator AI Agent with Google Drive Integration
# This agent generates university courses and manages them in Google Drive

import requests
from typing import Dict, List, Optional, Tuple
import json
import re
from datetime import datetime
import os
from google.colab import drive, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
from difflib import SequenceMatcher

class EnhancedCourseGeneratorAgent:
    def __init__(self, drive_folder_name: str = "AI_Generated_Courses"):
        self.name = "Enhanced Course Generator"
        self.description = "Generates university courses with Google Drive integration"
        self.capabilities = ["topic_research", "course_generation", "web_search", "drive_integration", "duplicate_detection", "trending_topics"]
        self.drive_folder_name = drive_folder_name
        self.drive_service = None
        self.folder_id = None
        self.existing_courses = []

        # Initialize Google Drive
        self._setup_drive()

    def _setup_drive(self):
        """Setup Google Drive API connection"""
        try:
            print("🔧 Setting up Google Drive connection...")

            # Mount Google Drive
            drive.mount('/content/drive')

            # Authenticate and build service
            auth.authenticate_user()
            self.drive_service = build('drive', 'v3')

            # Create or find the courses folder
            self._setup_courses_folder()

            # Load existing courses
            self._load_existing_courses()

            print("✅ Google Drive setup completed!")

        except Exception as e:
            print(f"❌ Drive setup failed: {e}")
            print("Note: Make sure to run this in Google Colab with Drive access")

    def _setup_courses_folder(self):
        """Create or find the courses folder in Google Drive"""
        try:
            # Search for existing folder
            query = f"name='{self.drive_folder_name}' and mimeType='application/vnd.google-apps.folder'"
            results = self.drive_service.files().list(q=query).execute()
            items = results.get('files', [])

            if items:
                self.folder_id = items[0]['id']
                print(f"📁 Found existing folder: {self.drive_folder_name}")
            else:
                # Create new folder
                folder_metadata = {
                    'name': self.drive_folder_name,
                    'mimeType': 'application/vnd.google-apps.folder'
                }
                folder = self.drive_service.files().create(body=folder_metadata).execute()
                self.folder_id = folder.get('id')
                print(f"📁 Created new folder: {self.drive_folder_name}")

        except Exception as e:
            print(f"❌ Folder setup failed: {e}")

    def _load_existing_courses(self):
        """Load all existing courses from the Drive folder"""
        try:
            if not self.folder_id:
                return

            print("📚 Loading existing courses...")

            # Search for JSON files in the folder
            query = f"'{self.folder_id}' in parents and name contains '.json'"
            results = self.drive_service.files().list(q=query).execute()
            files = results.get('files', [])

            self.existing_courses = []

            for file in files:
                try:
                    # Download and parse each course file
                    request = self.drive_service.files().get_media(fileId=file['id'])
                    file_content = io.BytesIO()
                    downloader = MediaIoBaseDownload(file_content, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()

                    # Parse JSON content
                    file_content.seek(0)
                    course_data = json.loads(file_content.read().decode('utf-8'))
                    self.existing_courses.append(course_data)

                except Exception as e:
                    print(f"⚠️ Could not load course file {file['name']}: {e}")

            print(f"✅ Loaded {len(self.existing_courses)} existing courses")

        except Exception as e:
            print(f"❌ Failed to load existing courses: {e}")

    def _similarity_score(self, text1: str, text2: str) -> float:
        """Calculate similarity between two texts"""
        return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

    def _find_similar_courses(self, topic: str, threshold: float = 0.3) -> List[Dict]:
        """Find courses similar to the given topic"""
        similar_courses = []

        for course in self.existing_courses:
            # Check similarity with topic, title, and description
            topic_sim = self._similarity_score(topic, course.get('topic', ''))
            title_sim = self._similarity_score(topic, course.get('title', ''))

            max_similarity = max(topic_sim, title_sim)

            if max_similarity > threshold:
                course_copy = course.copy()
                course_copy['similarity_score'] = max_similarity
                similar_courses.append(course_copy)

        # Sort by similarity (highest first)
        similar_courses.sort(key=lambda x: x['similarity_score'], reverse=True)
        return similar_courses

    def _generate_alternative_course(self, original_topic: str, similar_courses: List[Dict]) -> Dict[str, str]:
        """Generate an alternative course when similar ones exist"""

        # Analyze existing courses to determine what type of alternative to create
        existing_levels = []
        existing_topics = []

        for course in similar_courses:
            title = course.get('title', '').lower()
            topic = course.get('topic', '').lower()

            # Detect course levels
            if any(word in title for word in ['introduction', 'intro', 'basic', 'fundamentals']):
                existing_levels.append('beginner')
            elif any(word in title for word in ['advanced', 'expert', 'master']):
                existing_levels.append('advanced')
            else:
                existing_levels.append('intermediate')

            existing_topics.append(topic)

        # Decision logic for alternative course
        alternative_type = "advanced"  # default

        if 'beginner' in existing_levels and 'advanced' not in existing_levels:
            alternative_type = "advanced"
        elif 'advanced' in existing_levels and 'beginner' not in existing_levels:
            alternative_type = "beginner"
        elif len(similar_courses) >= 2:
            alternative_type = "specialized"

        return self._create_alternative_course(original_topic, alternative_type, similar_courses)

    def _create_alternative_course(self, topic: str, alt_type: str, similar_courses: List[Dict]) -> Dict[str, str]:
        """Create alternative course based on type"""

        if alt_type == "advanced":
            title = f"Advanced {topic}"
            description_intro = f"This advanced course builds upon foundational knowledge of {topic.lower()}"
            focus_areas = [
                "cutting-edge research and developments",
                "complex theoretical frameworks",
                "advanced methodologies and techniques",
                "critical analysis of contemporary issues",
                "independent research projects"
            ]
            prerequisites = f"Completion of introductory {topic.lower()} course or equivalent experience required."

        elif alt_type == "beginner":
            title = f"Introduction to {topic}"
            description_intro = f"This introductory course provides a gentle entry point into {topic.lower()}"
            focus_areas = [
                "fundamental concepts and terminology",
                "basic principles and applications",
                "historical context and development",
                "practical exercises and examples",
                "foundational skills development"
            ]
            prerequisites = "No prior knowledge required. Suitable for all students."

        else:  # specialized
            specializations = [
                f"{topic} in Healthcare Applications",
                f"{topic} for Business Innovation",
                f"{topic} and Society",
                f"Ethical Considerations in {topic}",
                f"{topic} Research Methods"
            ]
            title = specializations[len(similar_courses) % len(specializations)]
            description_intro = f"This specialized course explores the intersection of {topic.lower()} with specific domain applications"
            focus_areas = [
                "domain-specific applications",
                "interdisciplinary approaches",
                "case studies and real-world examples",
                "professional skills development",
                "industry collaboration projects"
            ]
            prerequisites = f"Basic understanding of {topic.lower()} recommended."

        # Build full description
        description_parts = [
            description_intro,
            f"Students will explore {', '.join(focus_areas[:3])}",
            "The course emphasizes both theoretical understanding and practical application through interactive learning experiences.",
            prerequisites
        ]

        description = ". ".join(description_parts) + "."

        return {
            "title": title,
            "description": description,
            "topic": topic,
            "course_type": alt_type,
            "related_courses": len(similar_courses),
            "generated_at": datetime.now().isoformat(),
            "alternative_reason": f"Similar courses detected - created {alt_type} alternative"
        }

    def get_trending_topics(self) -> List[str]:
        """Fetch trending topics from multiple sources"""
        trending_topics = []

        print("🔥 Fetching trending topics from the web...")

        try:
            # Method 1: Google Trends-like search (using DuckDuckGo for trending searches)
            trending_searches = [
                "trending technology 2025",
                "popular science topics",
                "current events education",
                "emerging fields study",
                "hot topics research"
            ]

            for search_term in trending_searches:
                try:
                    url = f"https://api.duckduckgo.com/?q={search_term}&format=json&no_html=1"
                    response = requests.get(url, timeout=5)

                    if response.status_code == 200:
                        data = response.json()

                        # Extract topics from related topics
                        if 'RelatedTopics' in data:
                            for item in data['RelatedTopics'][:3]:
                                if isinstance(item, dict) and 'Text' in item:
                                    text = item['Text']
                                    # Extract potential topics (words that might be course topics)
                                    words = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', text)
                                    trending_topics.extend(words[:2])

                except Exception as e:
                    continue

            # Method 2: Add some manually curated trending topics for 2025
            curated_trending = [
                "Artificial Intelligence Ethics",
                "Quantum Computing",
                "Climate Change Solutions",
                "Digital Transformation",
                "Blockchain Applications",
                "Cybersecurity Fundamentals",
                "Data Privacy Laws",
                "Remote Work Psychology",
                "Sustainable Technology",
                "Mental Health Awareness",
                "Social Media Impact",
                "Renewable Energy Systems",
                "Biotechnology Advances",
                "Space Technology",
                "Virtual Reality Applications",
                "Machine Learning Ethics",
                "Digital Marketing Trends",
                "Cryptocurrency Economics",
                "Green Architecture",
                "Telemedicine"
            ]

            trending_topics.extend(curated_trending)

            # Clean and deduplicate
            cleaned_topics = []
            seen = set()

            for topic in trending_topics:
                if isinstance(topic, str):
                    # Clean the topic
                    clean_topic = re.sub(r'[^\w\s]', '', topic).strip()
                    clean_topic = ' '.join(clean_topic.split())  # Remove extra spaces

                    # Filter valid topics (2-4 words, reasonable length)
                    if (2 <= len(clean_topic.split()) <= 4 and
                        5 <= len(clean_topic) <= 50 and
                        clean_topic.lower() not in seen):

                        cleaned_topics.append(clean_topic)
                        seen.add(clean_topic.lower())

            # Return top 15 (we'll show 10 but keep some extras)
            final_topics = cleaned_topics[:15]

            print(f"✅ Found {len(final_topics)} trending topics")
            return final_topics

        except Exception as e:
            print(f"❌ Failed to fetch trending topics: {e}")
            # Return fallback topics
            return [
                "Artificial Intelligence",
                "Climate Change",
                "Digital Marketing",
                "Cybersecurity",
                "Data Science",
                "Sustainable Energy",
                "Mental Health",
                "Blockchain Technology",
                "Virtual Reality",
                "Biotechnology"
            ]

    def display_trending_topics(self, topics: List[str]) -> Optional[str]:
        """Display trending topics and let user choose"""
        if not topics:
            print("❌ No trending topics available")
            return None

        print("\n🔥 TOP TRENDING TOPICS FOR COURSE GENERATION")
        print("="*50)

        # Show top 10
        display_topics = topics[:10]

        for i, topic in enumerate(display_topics, 1):
            print(f"{i:2d}. {topic}")

        print("\n" + "="*50)

        while True:
            try:
                choice = input("Select a topic (1-10) or 'r' to refresh: ").strip()

                if choice.lower() == 'r':
                    # Refresh trending topics
                    new_topics = self.get_trending_topics()
                    return self.display_trending_topics(new_topics)

                choice_num = int(choice)
                if 1 <= choice_num <= len(display_topics):
                    selected_topic = display_topics[choice_num - 1]
                    print(f"✅ Selected topic: {selected_topic}")
                    return selected_topic
                else:
                    print(f"Please enter a number between 1 and {len(display_topics)}")

            except ValueError:
                print("Please enter a valid number or 'r' to refresh")
            except KeyboardInterrupt:
                return None

    def search_topic_info(self, topic: str) -> str:
        """Retrieve information about a topic from the internet"""
        try:
            url = f"https://api.duckduckgo.com/?q={topic}&format=json&no_html=1&skip_disambig=1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()

                abstract = data.get('Abstract', '')
                definition = data.get('Definition', '')

                related_topics = []
                if 'RelatedTopics' in data:
                    for item in data['RelatedTopics'][:3]:
                        if isinstance(item, dict) and 'Text' in item:
                            related_topics.append(item['Text'])

                info = []
                if abstract:
                    info.append(f"Overview: {abstract}")
                if definition:
                    info.append(f"Definition: {definition}")
                if related_topics:
                    info.append(f"Related areas: {'; '.join(related_topics)}")

                return " | ".join(info) if info else f"Basic information about {topic}"

        except Exception as e:
            print(f"Web search failed: {e}")

        return f"General topic: {topic}"

    def generate_course_content(self, topic: str, topic_info: str = "") -> Dict[str, str]:
        """Generate standard course content"""

        title = f"Fundamentals of {topic}"

        description_parts = [
            f"This course provides a comprehensive examination of {topic.lower()}",
            f"Students will understand the fundamental principles of {topic.lower()}, analyze key concepts and methodologies, and evaluate contemporary applications",
            "The course combines theoretical foundations with practical applications, incorporating case studies and interactive discussions.",
            "Suitable for undergraduate students with basic academic preparation."
        ]

        description = ". ".join(description_parts) + "."

        return {
            "title": title,
            "description": description,
            "topic": topic,
            "course_type": "standard",
            "generated_at": datetime.now().isoformat(),
            "source_info": topic_info[:100] + "..." if len(topic_info) > 100 else topic_info
        }

    def _save_course_to_drive(self, course_data: Dict[str, str]) -> str:
        """Save course data to Google Drive"""
        try:
            if not self.drive_service or not self.folder_id:
                print("❌ Google Drive not properly configured")
                return None

            # Create filename
            safe_title = re.sub(r'[^\w\s-]', '', course_data['title'])
            safe_title = re.sub(r'[-\s]+', '_', safe_title)
            filename = f"{safe_title}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

            # Create file content
            file_content = json.dumps(course_data, indent=2)

            # Create temporary file path
            temp_file = f"/tmp/{filename}"

            # Write content to temporary file
            with open(temp_file, 'w', encoding='utf-8') as f:
                f.write(file_content)

            # Upload to Drive using the temporary file
            file_metadata = {
                'name': filename,
                'parents': [self.folder_id]
            }

            media = MediaFileUpload(
                temp_file,
                mimetype='application/json',
                resumable=True
            )

            file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media
            ).execute()

            # Clean up temporary file
            if os.path.exists(temp_file):
                os.remove(temp_file)

            print(f"💾 Course saved to Drive: {filename}")
            return file.get('id')

        except Exception as e:
            print(f"❌ Failed to save to Drive: {e}")
            # Clean up temp file in case of error
            temp_file = f"/tmp/{filename if 'filename' in locals() else 'temp_course'}.json"
            if os.path.exists(temp_file):
                os.remove(temp_file)
            return None

    def process_request(self, topic: str, use_web_search: bool = True) -> Dict[str, str]:
        """Main method to process a topic and generate course content"""
        print(f"🤖 Agent '{self.name}' processing topic: {topic}")

        # Step 1: Check for similar existing courses
        print("🔍 Checking for similar existing courses...")
        similar_courses = self._find_similar_courses(topic)

        if similar_courses:
            print(f"📚 Found {len(similar_courses)} similar courses:")
            for course in similar_courses[:3]:  # Show top 3
                print(f"  - {course['title']} (similarity: {course['similarity_score']:.2f})")

            print("🎯 Generating alternative course...")
            course_data = self._generate_alternative_course(topic, similar_courses)
        else:
            print("✨ No similar courses found, generating standard course...")

            # Step 2: Research topic if requested
            topic_info = ""
            if use_web_search:
                print("🔍 Searching for topic information...")
                topic_info = self.search_topic_info(topic)
                print(f"✅ Found: {topic_info[:100]}...")

            # Step 3: Generate course content
            course_data = self.generate_course_content(topic, topic_info)

        # Step 4: Save to Google Drive
        print("💾 Saving course to Google Drive...")
        file_id = self._save_course_to_drive(course_data)
        if file_id:
            course_data['drive_file_id'] = file_id
            # Add to existing courses list
            self.existing_courses.append(course_data)

        print("✅ Course generation completed!")
        return course_data

    def display_result(self, course_data: Dict[str, str]):
        """Display the generated course in a formatted way"""
        print("\n" + "="*60)
        print("📚 GENERATED UNIVERSITY COURSE")
        print("="*60)
        print(f"📖 TITLE: {course_data['title']}")
        print(f"🎯 TOPIC: {course_data['topic']}")
        print(f"🏷️ TYPE: {course_data.get('course_type', 'standard').upper()}")

        if 'alternative_reason' in course_data:
            print(f"💡 REASON: {course_data['alternative_reason']}")

        print(f"🕒 GENERATED: {course_data['generated_at']}")

        if 'drive_file_id' in course_data:
            print(f"💾 SAVED TO DRIVE: ✅")

        print("\n📝 DESCRIPTION:")
        print("-" * 40)
        print(course_data['description'])

        if 'source_info' in course_data:
            print("\n🔍 SOURCE INFO:")
            print("-" * 40)
            print(course_data['source_info'])

        print("="*60)

    def list_existing_courses(self):
        """Display all existing courses"""
        if not self.existing_courses:
            print("📚 No courses found in the database.")
            return

        print(f"\n📚 EXISTING COURSES ({len(self.existing_courses)} total)")
        print("="*60)

        for i, course in enumerate(self.existing_courses, 1):
            print(f"{i}. {course['title']}")
            print(f"   Topic: {course['topic']} | Type: {course.get('course_type', 'standard')}")
            print(f"   Generated: {course['generated_at'][:10]}")
            print()

# Main function for demonstration
def main():
    """Main function to demonstrate the enhanced agent"""
    print("🚀 Starting Enhanced Course Generator Agent")
    print("This agent now integrates with Google Drive!")

    # Initialize the agent (will setup Drive connection)
    agent = EnhancedCourseGeneratorAgent()

    if not agent.drive_service:
        print("❌ Could not connect to Google Drive. Please check your setup.")
        return

    # Interactive mode
    while True:
        print("\n" + "="*50)
        print("ENHANCED COURSE GENERATOR AGENT")
        print("="*50)
        print("1. Generate course for new topic")
        print("2. Generate course from trending topics 🔥")
        print("3. View existing courses")
        print("4. Test with sample topics")
        print("5. Generate without web search")
        print("6. Exit")

        choice = input("\nSelect option (1-6): ").strip()

        if choice == "1":
            topic = input("Enter your topic: ").strip()
            if topic:
                course_data = agent.process_request(topic, use_web_search=True)
                agent.display_result(course_data)

        elif choice == "2":
            print("🔥 Fetching trending topics for course generation...")
            trending_topics = agent.get_trending_topics()

            if trending_topics:
                selected_topic = agent.display_trending_topics(trending_topics)

                if selected_topic:
                    print(f"\n🎯 Generating course for trending topic: {selected_topic}")
                    course_data = agent.process_request(selected_topic, use_web_search=True)
                    agent.display_result(course_data)
                else:
                    print("❌ No topic selected")
            else:
                print("❌ Could not fetch trending topics")

        elif choice == "3":
            agent.list_existing_courses()

        elif choice == "4":
            test_topics = [
                "Machine Learning",
                "Machine Learning",  # Duplicate to test alternative generation
                "Deep Learning",     # Similar to test alternative generation
                "Sustainable Energy",
                "Digital Marketing"
            ]

            print("🧪 Running test with sample topics...")
            for topic in test_topics:
                print(f"\n--- Processing: {topic} ---")
                course_data = agent.process_request(topic)
                agent.display_result(course_data)
                input("\nPress Enter to continue to next topic...")

        elif choice == "5":
            topic = input("Enter your topic (no web search): ").strip()
            if topic:
                course_data = agent.process_request(topic, use_web_search=False)
                agent.display_result(course_data)

        elif choice == "6":
            print("👋 Goodbye! Agent shutting down.")
            break

        else:
            print("Invalid option! Please try again.")

# Quick test function for Colab
def quick_test():
    """Quick test without interactive mode - good for Colab"""
    print("🧪 Quick Test Mode")

    agent = EnhancedCourseGeneratorAgent()

    if not agent.drive_service:
        print("❌ Could not connect to Google Drive")
        return

    # Test with sample topics
    test_topics = ["Artificial Intelligence", "AI Ethics", "Machine Learning"]

    for topic in test_topics:
        print(f"\n--- Testing with: {topic} ---")
        course_data = agent.process_request(topic)
        agent.display_result(course_data)

    print(f"\n📊 Total courses in database: {len(agent.existing_courses)}")

# For immediate testing
if __name__ == "__main__":
    main()

# Uncomment to run quick test
# quick_test()

🚀 Starting Enhanced Course Generator Agent
This agent now integrates with Google Drive!
🔧 Setting up Google Drive connection...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Found existing folder: AI_Generated_Courses
📚 Loading existing courses...
✅ Loaded 6 existing courses
✅ Google Drive setup completed!

ENHANCED COURSE GENERATOR AGENT
1. Generate course for new topic
2. Generate course from trending topics 🔥
3. View existing courses
4. Test with sample topics
5. Generate without web search
6. Exit

Select option (1-6): 3

📚 EXISTING COURSES (6 total)
1. Ethical Considerations in Sustainable Energy
   Topic: Sustainable Energy | Type: specialized
   Generated: 2025-06-21

2. Ethical Considerations in Digital Marketing
   Topic: Digital Marketing | Type: specialized
   Generated: 2025-06-21

3. Sustainable Energy and Society
   Topic: Sustainable Energy | Type: specialized
   Generated: 2025-06-21

4